In [14]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras import backend as K
import pre_processing as pp
from importlib import reload
K.set_image_dim_ordering('th')

In [15]:
training = pp.get_training()

In [24]:
test = pp.get_test()

In [25]:
X_train = np.array([x for x, y in training])
y_train = np.array([y for x, y in training])
X_test = test

In [27]:
pixel_count = X_train.shape[1] * X_train.shape[2]

In [28]:
# Normalize values (0-1)
X_train = np.divide(X_train, 255)
X_test = np.divide(X_test, 255)
y_train = np_utils.to_categorical(y_train)

In [29]:
class_count = y_train.shape[1]

In [30]:
# Seed for reproducibility
seed = 42
np.random.seed(seed)

In [12]:
# Flatten
# X_train = X_train.reshape(X_train.shape[0], pixel_count).astype('float32')
# X_test = X_test.reshape(X_test.shape[0], pixel_count).astype('float32')

In [13]:
# BASE NEURAL NETWORK MODEL
# def base_nn():
#     nn_model = Sequential()
#     # Relu hidden layer, same number of neurons as input layer
#     nn_model.add(Dense(pixel_count, input_dim=pixel_count, kernel_initializer='normal', activation='relu'))
#     # Softmax output layer
#     nn_model.add(Dense(class_count, kernel_initializer='normal', activation='softmax'))
#     # Compile
#     nn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return nn_model

In [14]:
# # Create a base neural network model
# model = base_nn()
# model.fit(X_train, y_train_vec, validation_data=(X_test, y_test_vec), epochs=10, batch_size=50, verbose=2)
# scores = model.evaluate(X_test, y_test_vec, verbose=0)
# print("Baseline Error %.2f%%" % (100-scores[1]*100))

In [31]:
# FOR CNN
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')

In [16]:
def base_cnn():
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(class_count, activation='softmax'))
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [32]:
def larger_cnn():
    # create model
    model = Sequential()
    model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(15, (3, 3), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(50, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(class_count, activation='softmax'))
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
def deep_cnn():
    # create model
    model = Sequential()
    model.add(Conv2D(1, (28, 28), input_shape=(1, 28, 28), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Conv2D(512, (24, 24), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(256, (20, 20), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    model.add(Conv2D(128, (16, 16), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (8, 8), activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))
    model.add(Flatten())
    model.add(Dense(2048, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dense(1024, activation='linear'))
    model.add(LeakyReLU(alpha=0.001))
    model.add(Dropout(0.1))
    model.add(Dense(class_count, activation='softmax'))
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
clf = deep_cnn()
clf.fit(X_train, y_train, epochs=400, batch_size=200, verbose=2)

Epoch 1/400
 - 151s - loss: 1.9600 - acc: 0.2823
Epoch 2/400
 - 149s - loss: 1.3592 - acc: 0.5360
Epoch 3/400
 - 150s - loss: 1.0513 - acc: 0.6586
Epoch 4/400
 - 150s - loss: 0.8821 - acc: 0.7192
Epoch 5/400
 - 151s - loss: 0.7778 - acc: 0.7564
Epoch 6/400
 - 151s - loss: 0.6981 - acc: 0.7832
Epoch 7/400
 - 149s - loss: 0.6574 - acc: 0.7972
Epoch 8/400
 - 148s - loss: 0.6065 - acc: 0.8149
Epoch 9/400
 - 150s - loss: 0.5788 - acc: 0.8226
Epoch 10/400
 - 149s - loss: 0.5445 - acc: 0.8334
Epoch 11/400
 - 149s - loss: 0.5299 - acc: 0.8377
Epoch 12/400
 - 149s - loss: 0.5131 - acc: 0.8434
Epoch 13/400
 - 149s - loss: 0.4869 - acc: 0.8505
Epoch 14/400
 - 150s - loss: 0.4720 - acc: 0.8570
Epoch 15/400
 - 150s - loss: 0.4632 - acc: 0.8598
Epoch 16/400
 - 150s - loss: 0.4505 - acc: 0.8626
Epoch 17/400
 - 149s - loss: 0.4351 - acc: 0.8686
Epoch 18/400
 - 150s - loss: 0.4287 - acc: 0.8681
Epoch 19/400
 - 149s - loss: 0.4159 - acc: 0.8732
Epoch 20/400
 - 150s - loss: 0.4103 - acc: 0.8741
Epoch 21/

Epoch 164/400
 - 148s - loss: 0.1731 - acc: 0.9421
Epoch 165/400
 - 150s - loss: 0.1684 - acc: 0.9426
Epoch 166/400
 - 149s - loss: 0.1674 - acc: 0.9432
Epoch 167/400
 - 148s - loss: 0.1754 - acc: 0.9421
Epoch 168/400
 - 149s - loss: 0.1714 - acc: 0.9415
Epoch 169/400
 - 150s - loss: 0.1672 - acc: 0.9434
Epoch 170/400
 - 150s - loss: 0.1715 - acc: 0.9424
Epoch 171/400
 - 150s - loss: 0.1685 - acc: 0.9431
Epoch 172/400
 - 149s - loss: 0.1697 - acc: 0.9427
Epoch 173/400
 - 149s - loss: 0.1635 - acc: 0.9437
Epoch 174/400
 - 147s - loss: 0.1661 - acc: 0.9432
Epoch 175/400
 - 147s - loss: 0.1744 - acc: 0.9408
Epoch 176/400
 - 149s - loss: 0.1667 - acc: 0.9432
Epoch 177/400
 - 148s - loss: 0.1677 - acc: 0.9435
Epoch 178/400
 - 147s - loss: 0.1661 - acc: 0.9434
Epoch 179/400
 - 148s - loss: 0.1673 - acc: 0.9435
Epoch 180/400
 - 148s - loss: 0.1650 - acc: 0.9445
Epoch 181/400
 - 149s - loss: 0.1655 - acc: 0.9451
Epoch 182/400
 - 150s - loss: 0.1698 - acc: 0.9416
Epoch 183/400
 - 150s - loss: 0

Epoch 325/400
 - 150s - loss: 0.1451 - acc: 0.9517
Epoch 326/400
 - 150s - loss: 0.1409 - acc: 0.9522
Epoch 327/400
 - 148s - loss: 0.1358 - acc: 0.9544
Epoch 328/400
 - 151s - loss: 0.1419 - acc: 0.9518
Epoch 329/400
 - 149s - loss: 0.1387 - acc: 0.9534
Epoch 330/400
 - 150s - loss: 0.1439 - acc: 0.9520
Epoch 331/400
 - 150s - loss: 0.1420 - acc: 0.9526
Epoch 332/400
 - 150s - loss: 0.1387 - acc: 0.9534
Epoch 333/400
 - 148s - loss: 0.1405 - acc: 0.9529
Epoch 334/400
 - 149s - loss: 0.1402 - acc: 0.9527
Epoch 335/400
 - 149s - loss: 0.1395 - acc: 0.9533
Epoch 336/400
 - 149s - loss: 0.1346 - acc: 0.9547
Epoch 337/400
 - 148s - loss: 0.1385 - acc: 0.9528
Epoch 338/400
 - 149s - loss: 0.1441 - acc: 0.9516
Epoch 339/400
 - 149s - loss: 0.1326 - acc: 0.9556
Epoch 340/400
 - 149s - loss: 0.1370 - acc: 0.9546
Epoch 341/400
 - 148s - loss: 0.1400 - acc: 0.9534
Epoch 342/400
 - 148s - loss: 0.1379 - acc: 0.9538
Epoch 343/400
 - 150s - loss: 0.1396 - acc: 0.9531
Epoch 344/400
 - 149s - loss: 0

In [56]:
preds = clf.predict(X_test)

In [57]:
pred_nums = np.array([np.argmax(pred) for pred in preds])

In [58]:
pred_enum = np.array([(i, x) for (i, x) in enumerate(pred_nums)])

In [59]:
np.savetxt('../data/test_y.csv', pred_enum, delimiter=',', fmt='%d', header='Id,Label')